In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/freesound-audio-tagging/sample_submission.csv
/kaggle/input/freesound-audio-tagging/train_post_competition.csv
/kaggle/input/freesound-audio-tagging/train.csv
/kaggle/input/freesound-audio-tagging/test_post_competition.csv
/kaggle/input/freesound-audio-tagging/audio_train/66dd3d3b.wav
/kaggle/input/freesound-audio-tagging/audio_train/edf36df4.wav
/kaggle/input/freesound-audio-tagging/audio_train/6496bf6a.wav
/kaggle/input/freesound-audio-tagging/audio_train/cc046f8d.wav
/kaggle/input/freesound-audio-tagging/audio_train/34881c5f.wav
/kaggle/input/freesound-audio-tagging/audio_train/860a2f2e.wav
/kaggle/input/freesound-audio-tagging/audio_train/10086933.wav
/kaggle/input/freesound-audio-tagging/audio_train/e94c6d67.wav
/kaggle/input/freesound-audio-tagging/audio_train/503e81d0.wav
/kaggle/input/freesound-audio-tagging/audio_train/3b04a2b1.wav
/kaggle/input/freesound-audio-tagging/audio_train/fc950d92.wav
/kaggle/input/freesound-audio-tagging/audio_train/9e969e02.wav
/kaggle

In [2]:
train_ds = pd.read_csv("/kaggle/input/freesound-audio-tagging/train.csv")
test_ds = pd.read_csv("/kaggle/input/freesound-audio-tagging/test_post_competition.csv")
sample_sub = pd.read_csv("/kaggle/input/freesound-audio-tagging/sample_submission.csv")

In [3]:
train_ds.columns

Index(['fname', 'label', 'manually_verified'], dtype='object')

In [4]:
path = "/kaggle/input/freesound-audio-tagging/audio_"

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import AutoTokenizer, HubertForSequenceClassification, AutoFeatureExtractor
import librosa
from tqdm import tqdm
from transformers import AutoProcessor

2026-02-20 09:18:56.847391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771579137.099476      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771579137.165403      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771579137.747696      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771579137.747742      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771579137.747746      55 computation_placer.cc:177] computation placer alr

In [6]:
torch.cuda.set_per_process_memory_fraction(0.5, device=0)

In [7]:
def extract(file):
    audio , sample_rate = librosa.load(file, sr = 16000)
    return audio, sample_rate

In [8]:
train_data = []
sample_rate = 0
for idx, row in  tqdm(train_ds.iterrows()):
    file =os.path.join(path + "train", row["fname"])
    audio, sample_rate = extract(file)
    train_data.append(audio)
    sample_rate = sample_rate

9473it [02:55, 53.92it/s]


In [9]:
test_data=[]
sample_rate_test =0
for idx , row in tqdm(test_ds.iterrows()):
    file = os.path.join(path + "test", row["fname"])
    audio, sample_rate= extract(file)
    test_data.append(audio)
    sample_rate_test =sample_rate

9400it [02:30, 62.37it/s]


In [10]:
from sklearn.preprocessing import LabelEncoder

encoder= LabelEncoder()
train_ds["label"] = encoder.fit_transform(train_ds["label"])

In [11]:
processor = AutoFeatureExtractor.from_pretrained(
    "facebook/hubert-base-ls960"
)

preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

In [12]:
train_data[1]

array([-6.3894142e-04, -1.1385819e-03, -1.1121513e-03, ...,
        3.8902181e-06, -7.2033909e-05, -2.2096296e-05], dtype=float32)

In [13]:
class AudioDataset(Dataset):
    def __init__(self, ds, audio_data, is_test=False):
        self.audio = audio_data
        self.is_test = is_test

        if not is_test:
            self.labels = ds["label"].values

    def __len__(self):
        return len(self.audio)

    def __getitem__(self, idx):
        audio = self.audio[idx]

        if isinstance(audio, list):
            audio = np.array(audio)

        if len(audio.shape) == 2:
            audio = audio.squeeze()

        audio = audio.astype(np.float32)

        if not self.is_test:
            return audio, self.labels[idx]
        else:
            return audio

In [14]:
def collate_fn(batch):
    if isinstance(batch[0], tuple):
        audios, labels = zip(*batch)
    else:
        audios = batch
        labels = None

    inputs = processor(
        list(audios),
        sampling_rate=16000,
        padding=True,
        truncation=True,
        max_length=16000 * 5,   # 5 seconds max
        return_tensors="pt",
        return_attention_mask=True
    )
    if labels is not None:
        inputs["labels"] = torch.tensor(labels, dtype=torch.long)

    return inputs

In [15]:
train_df = AudioDataset(train_ds,train_data, is_test =False)
test_df=AudioDataset(test_ds, test_data, is_test =True)

train_dl = DataLoader(
    train_df,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers =4,
    pin_memory =True
)

test_dl = DataLoader(
    test_df,
    batch_size=2,
    shuffle=False,
    collate_fn=collate_fn
)


In [17]:
device = ("cuda" if torch.cuda.is_available()else "cpu")

In [18]:
for batch in train_dl:
    input_values = batch["input_values"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    y = batch["labels"].to(device)
    break

In [21]:
import torch.nn as nn
from transformers import HubertForSequenceClassification

class Model(nn.Module):
    def __init__(self, target_size):
        super().__init__() 
        
        self.model = HubertForSequenceClassification.from_pretrained(
            "facebook/hubert-base-ls960",
            num_labels=target_size,
            ignore_mismatched_sizes=True
        )
        for param in self.model.hubert.parameters():
            param.requires_grad = False
            
    def forward(self, input_values, attention_mask=None):
        outputs = self.model(
            input_values=input_values,
            attention_mask=attention_mask
        )    
        return outputs.logits

In [22]:
target_size = train_ds["label"].nunique()
device =("cuda" if torch.cuda.is_available() else "cpu")
model = Model(target_size= target_size ).to(device)
print(model)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model(
  (model): HubertForSequenceClassification(
    (hubert): HubertModel(
      (feature_extractor): HubertFeatureEncoder(
        (conv_layers): ModuleList(
          (0): HubertGroupNormConvLayer(
            (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
            (activation): GELUActivation()
            (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
          )
          (1-4): 4 x HubertNoLayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (activation): GELUActivation()
          )
          (5-6): 2 x HubertNoLayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
            (activation): GELUActivation()
          )
        )
      )
      (feature_projection): HubertFeatureProjection(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (projection): Linear(in_features=512, out_features=768, bias=True)
   

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=2e-6)

scaler = torch.amp.GradScaler("cuda")

model.to(device)

for epoch in range(10):
    model.train()
    t_loss = 0

    for batch in train_dl:
        optimizer.zero_grad()

        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.amp.autocast("cuda"):
            logits = model(
                batch["input_values"],
                batch["attention_mask"]
            )
            loss = criterion(logits, batch["labels"])

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        t_loss += loss.item()

    print(f"Epoch {epoch+1}")
    print("Loss:", t_loss / len(train_dl))

Epoch 1
Loss: 3.606888548593519
Epoch 2
Loss: 3.3662792783209308


In [ ]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model.eval()
with